Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X.shape

(150, 4)

In [3]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [4]:
def gb_predict(X, trees_list, coef_list, eta):
    return np.array([sum([eta* coef * alg.predict([x])[0] for alg, coef in zip(trees_list, coef_list)]) for x in X])

def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)

def bias(y, z):
    return (y - z)

In [5]:
def gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta):
    
    trees = []
    
    train_errors = []
    test_errors = []
    
    for i in range(n_trees):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)
        
        if len(trees) == 0:
            tree.fit(X_train, y_train)
            
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, coefs, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, coefs, eta)))
        else:
            target = gb_predict(X_train, trees, coefs, eta)
            tree.fit(X_train, bias(y_train, target))
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, coefs, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, coefs, eta)))

        trees.append(tree)
        
    return trees, train_errors, test_errors

In [6]:
n_trees = 25

coefs = [1] * n_trees

max_depth = 5

eta = 0.1

trees, train_errors, test_errors = gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta)

In [7]:
def evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta):
    train_prediction = gb_predict(X_train, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тренировочной выборке: {mean_squared_error(y_train, train_prediction)}')

    test_prediction = gb_predict(X_test, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тестовой выборке: {mean_squared_error(y_test, test_prediction)}')

In [8]:
evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta)

Ошибка алгоритма из 25 деревьев глубиной 5     с шагом 0.1 на тренировочной выборке: 0.00860496396222196
Ошибка алгоритма из 25 деревьев глубиной 5     с шагом 0.1 на тестовой выборке: 0.10979944928042681


Снизим размерность и проверим данные на аналогичной модели.

In [9]:
X_ = X.astype(float)

rows, cols = X_.shape

means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [10]:
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

eig_pairs.sort(key=lambda x: x[0], reverse=True)



Таким образом, первая главная компонента описывает почти 73% информации, а первые две в сумме - 95.8%. В то же время последняя компонента описывает всего 0.5% и может быть отброжена без страха значительных потерь в качестве нашего анализа. Мы отбросим последние две компоненты, оставив первые две.

In [11]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [12]:
# Сформируем новую матрицу "объекты-признаки"
Z = X_.dot(W)

In [13]:
Z.shape

(150, 2)

In [14]:
Z_train, Z_test, y_train, y_test = model_selection.train_test_split(Z, y, test_size=0.25)

In [15]:
n_trees = 25

coefs = [1] * n_trees

max_depth = 5

eta = 0.1

trees, train_errors, test_errors = gb_fit(n_trees, max_depth, Z_train, Z_test, y_train, y_test, coefs, eta)

In [16]:
evaluate_alg(Z_train, Z_test, y_train, y_test, trees, coefs, eta)

Ошибка алгоритма из 25 деревьев глубиной 5     с шагом 0.1 на тренировочной выборке: 0.009274144539275447
Ошибка алгоритма из 25 деревьев глубиной 5     с шагом 0.1 на тестовой выборке: 0.1387052887923818


При построении модели со сниженными данными (2 компоненты), качетво модели на тестовой выборке незначительно выросло (разница = 0,02)

Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [17]:
#?np.linalg.svd

In [18]:
iris = datasets.load_iris()
X = iris.data

In [19]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [20]:
U, D, W = np.linalg.svd(X_)

V = W.T

In [21]:
print(V)

[[ 0.52106591 -0.37741762  0.71956635  0.26128628]
 [-0.26934744 -0.92329566 -0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161 -0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199 -0.63427274  0.52359713]]


In [22]:
print(D)

[20.92306556 11.7091661   4.69185798  1.76273239]


In [23]:
Z_new = X_.dot(V[:, :2])

In [24]:
Z_new[:10]

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356]])

In [25]:
Z_new.shape

(150, 2)

In [26]:
Z_new_train, Z_new_test, y_train, y_test = model_selection.train_test_split(Z_new, y, test_size=0.25)

In [27]:
n_trees = 25

coefs = [1] * n_trees

max_depth = 5

eta = 0.1

trees, train_errors, test_errors = gb_fit(n_trees, max_depth, Z_new_train, Z_new_test, y_train, y_test, coefs, eta)

In [28]:
evaluate_alg(Z_new_train, Z_new_test, y_train, y_test, trees, coefs, eta)

Ошибка алгоритма из 25 деревьев глубиной 5     с шагом 0.1 на тренировочной выборке: 0.010400315300638072
Ошибка алгоритма из 25 деревьев глубиной 5     с шагом 0.1 на тестовой выборке: 0.0705543297684191
